In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from os import listdir
import random
import math
import numpy as np



In [2]:
class preprocess(object):
        def __init__(self, dataset_location="", batch_size=1,shuffle=False):

            self.location=dataset_location
            self.idx =None
            self.batch_size=batch_size
            self.length=None
            self.seed =None 
            self.shuffle =shuffle
        def rescale(self,s):
            k = self.__getitem__()

            def bilinear_resize(image, height, width):

                img_height, img_width = image.shape[:2]

                resized = np.empty([height, width])

                x_ratio = float(img_width - 1) / (width - 1) if width > 1 else 0
                y_ratio = float(img_height - 1) / (height - 1) if height > 1 else 0

                for i in range(height):
                    for j in range(width):

                        x_l, y_l = math.floor(x_ratio * j), math.floor(y_ratio * i)
                        x_h, y_h = math.ceil(x_ratio * j), math.ceil(y_ratio * i)

                        x_weight = (x_ratio * j) - x_l
                        y_weight = (y_ratio * i) - y_l

                        a = image[y_l, x_l]
                        b = image[y_l, x_h]
                        c = image[y_h, x_l]
                        d = image[y_h, x_h]
                        pixel = a * (1 - x_weight) * (1 - y_weight)+ b * x_weight * (1 - y_weight)+c * y_weight * (1 - x_weight) + d * x_weight * y_weight
                        resized[i][j] = pixel

                return resized
            def bilinear_resize_1(image, height, width):

                img_height, img_width,img_x = image.shape[:]

                resized = np.empty([height, width,img_x])

                x_ratio = float(img_width - 1) / (width - 1) if width > 1 else 0
                y_ratio = float(img_height - 1) / (height - 1) if height > 1 else 0

                for i in range(height):
                    for j in range(width):

                        x_l, y_l = math.floor(x_ratio * j), math.floor(y_ratio * i)
                        x_h, y_h = math.ceil(x_ratio * j), math.ceil(y_ratio * i)

                        x_weight = (x_ratio * j) - x_l
                        y_weight = (y_ratio * i) - y_l

                        a = image[y_l, x_l]
                        b = image[y_l, x_h]
                        c = image[y_h, x_l]
                        d = image[y_h, x_h]
                        pixel = a * (1 - x_weight) * (1 - y_weight)+ b * x_weight * (1 - y_weight)+c * y_weight * (1 - x_weight) + d * x_weight * y_weight
                        resized[i,j,:] = pixel

                return resized
            p = {}
            for i in k.keys():
                if np.ndim(k[i]) == 2:
                    p[i] = bilinear_resize(k[i],s*len(k[i]),s*len(k[i].T))
                else:
                    p[i] = bilinear_resize_1(k[i],s*len(k[i]),s*len(k[i].T))
            return p 
        def edge_detection(self):
            k = self.rgb2gray()

            def edge_detection_1(m):
                frame = m
                q = np.copy(frame)
                r = np.copy(frame)
                G1 = np.array([[-1,0,1,-2,0,2,-1,0,1]])
                G2 = np.array([[1,2,1,0,0,0,-1,-2,-1]])
                for _ in range(1):
                    r1,c1 = np.shape(frame)
                    l = np.zeros((r1+2,c1+2))
                    l[1:-1,1:-1] = frame
                    for i in range(2, len(l) - 2):
                        for j in range(2, len(l.T) - 2):
                            h = np.array([l[i-1][j-1],l[i-1][j],l[i-1][j+1],l[i][j-1],l[i][j],l[i][j+1],l[i+1][j-1],l[i+1][j],l[i+1][j+1]])
                            q[i-2][j-2] = np.matmul(G1,h.T)
                            r[i-2][j-2] = np.matmul(G2,h.T)
                b = np.full([len(q),len(q.T)], 0)
                for i in range(len(q)):
                    for j in range(len(q.T)):
                        b[i][j] = round(math.sqrt(((q[i][j])**2)+((r[i][j])**2)))
                return b

            jp = {} 
            for le in k.keys():
                jp[le] = edge_detection_1(k[le])
            return jp


        def rotate(self,theta):
            k = self.__getitem__()
            def get_rotated_array(image,theta): 
                theta=math.radians(theta)                               
                h,w = np.shape(image)
                original_ch = round(((h+1)/2)-1)   
                original_cw = round(((w+1)/2)-1) 
                new_h=round(abs(h*math.cos(theta))+abs(w*math.sin(theta)))+1
                new_w=round(abs(w*math.cos(theta))+abs(h*math.sin(theta)))+1
                new_ch= round(((new_h+1)/2)-1)       
                new_cw= round(((new_w+1)/2)-1)
                output = np.zeros((new_h,new_w))
                for i in range(h):
                    for j in range(w):
                        y=image.shape[0]-1-i-original_ch                
                        x=image.shape[1]-1-j-original_cw   
                        new_y=round(-x*math.sin(theta)+y*math.cos(theta))
                        new_x=round(x*math.cos(theta)+y*math.sin(theta))
                        new_y=new_ch-new_y
                        new_x=new_cw-new_x
                        if 0 <= new_x < new_w and 0 <= new_y < new_h:
                            output[new_y,new_x]=image[i,j] 
                return output
            def get_rotated_array1(image,theta): 
                theta=math.radians(theta)                               
                h,w,hh= np.shape(image)
                original_ch = round(((h+1)/2)-1)   
                original_cw = round(((w+1)/2)-1) 
                new_h=round(abs(h*math.cos(theta))+abs(w*math.sin(theta)))+1
                new_w=round(abs(w*math.cos(theta))+abs(h*math.sin(theta)))+1
                new_ch= round(((new_h+1)/2)-1)       
                new_cw= round(((new_w+1)/2)-1)
                output = np.zeros((new_h,new_w,hh))
                for i in range(h):
                    for j in range(w):
                        y=image.shape[0]-1-i-original_ch                
                        x=image.shape[1]-1-j-original_cw   
                        new_y=round(-x*math.sin(theta)+y*math.cos(theta))
                        new_x=round(x*math.cos(theta)+y*math.sin(theta))
                        new_y=new_ch-new_y
                        new_x=new_cw-new_x
                        if 0 <= new_x < new_w and 0 <= new_y < new_h:
                            output[new_y,new_x,:]=image[i,j] 
                return output
            p = {}
            for i in k.keys():
                if np.ndim(k[i]) == 2:
                    p[i] = get_rotated_array(k[i],theta)
                else:
                    p[i] = get_rotated_array1(k[i],theta)
            return p



        def translate(self,tx,ty):
            k = self.__getitem__()
            oo = {}
            for g in k.keys(): 
                if np.ndim(k[g]) == 2:
                    row,column = np.shape(k[g])
                    ii = np.zeros((row + abs(ty),column + abs(tx)))
                    if tx > 0 and ty >0:
                        ii[:row,tx:column+tx] = k[g]
                    elif tx > 0 and ty < 0:
                        ii[-1*ty :row - ty ,tx:column+tx] = k[g]
                    elif tx < 0 and ty < 0:
                        ii[-1*ty:-1*ty+row,:column] = k[g]
                    elif tx < 0 and ty >0:
                        ii[:row,:column] = k[g]
                    else:
                        ii = k[g]
                    oo[g] = ii
                else:
                    row,column,height = np.shape(k[g])
                    ii = np.zeros((row + ty,column + tx,height))
                    ii[:row,tx:column+tx,:] = k[g]
                    if tx > 0 and ty >0:
                        ii[:row,tx:column+tx,:] = k[g]
                    elif tx > 0 and ty < 0:
                        ii[-1*ty :row - ty ,tx:column+tx,:] = k[g]
                    elif tx < 0 and ty < 0:
                        ii[-1*ty:-1*ty+row,:column,:] = k[g]
                    elif tx < 0 and ty >0:
                        ii[:row,:column,:] = k[g]
                    else:
                        ii = k[g]
                    oo[g] = ii
                    oo[g] = ii

            return oo

        def resize(self,h,w):
            h_new=h
            w_new=w
            dic , imgs ={} , self.__getitem__()
            for k in imgs:
                arr= imgs[k]
                height,width=arr.shape[:2]
                h_ratio=h_new/(height)
                w_ratio=w_new/(width)
                if len(arr.shape)==2:
                    req = np.zeros((h_new, w_new), np.uint8)
                    for i in range(h_new):
                        for j in range(w_new):
                            h_req = int(i / h_ratio)
                            w_req = int(j / w_ratio)
                            req[i, j] = arr[h_req, w_req]
                else:
                    req=np.zeros((h_new,w_new,3),np.uint8)
                    for i in range(h_new):
                        for j in range(w_new):
                            for t in range(3):
                                h_req = int(i / h_ratio)
                                w_req = int(j / w_ratio)
                                req[i,j,t] = arr[h_req,w_req,t]
                # Image.fromarray(req).show()
                dic[k]=req
            return dic

    #     # Resizes the batch of input images to the given dimensions. Where h is the number of rows and w is the number of columns
    #     # This function has to return a dictionary of resized images where the key is the name of the image and the value is corresponding resized output in accordance to the random indexes picked up.
    #     # The datatype of output has to be a numpy array

        def __getitem__(self):
            l=os.listdir(self.location)
            x=[]
            x.append(l[10])
            x=x+l[12:]+l[:10]
            x.append(l[11])
            dict={}
            if(self.shuffle==True):
                for i in range(self.batch_size):
                    
                    k=random.randint(0,19)
                    
                    image=plt.imread(self.location+x[k])
                        
                    dict[x[k][:-4]]=image
            else:
                for i in range(self.batch_size):
                    image=plt.imread(self.location+x[i])
                    
                    dict[x[i][:-4]]=image   
            return(dict)
        
  
        
        def crop(self,id1,id2,id3,id4):
            crop_dic={}
            imgdir=self.__getitem__()
            for k in imgdir:
                im=imgdir[k]
         
         
                if(len(im.shape)==2):
                    arr = im[id1[0]:(id2[0]+1),id1[1]:(id4[1]+1)]
                    crop_dic[k]=arr
                else:
                    arr = im[id1[0]:(id2[0]+1),id1[1]:(id4[1]+1),:3]
                    crop_dic[k]=arr
            return crop_dic
    #         def translate(self,tx,ty):
    #             dict=self.__getitem__()
    #             for ele in dict:
               

        def blur(self):
            k = self.__getitem__()
    #         print(k)
            cc = []
            for i in k.keys():
                ll = np.zeros((len(k[i])+2,len(k[i].T)+2))
    #             print(ll.shape,k[i].shape)
    
                ll[1:-1,1:-1] = k[i]
    #             print(ll)
                cc.append(ll)
    #         print(cc)
            def blurring_array(a):
                ker = np.array([[1.0,1.0,1.0], [1.0,1.0,1.0], [1.0,1.0,1.0]])
    #             kernel = kernel / np.sum(kernel)
                arraylist = []
                for y in range(3):
                    temparray = np.copy(a)
                    temparray = np.roll(temparray, y - 1, axis=0)
                    for x in range(3):
                        
                        temparray_X = np.copy(temparray)
                        temparray_X = np.roll(temparray_X, x - 1, axis=1)*ker[y,x]
#                         print(temparray_X)
                        arraylist.append(temparray_X)
    

                arraylist = np.array(arraylist)
                arraylist_median = np.median(arraylist, axis=0)
                
                return arraylist_median

                return ff
            jo = []
            for m in cc:
                p = blurring_array(m) 
                jo.append(p)
#             print(plt.imshow(jo[0]))


            return (dict(zip(k.keys(),jo)))
          
        def rgb2gray(self):
            j = self.__getitem__()
            gray = []
            for i  in j.keys(): 
                if j[i].ndim >2: 
                    r, g, b = j[i][:,:,0], j[i][:,:,1], j[i][:,:,2]
                    gray_scale = 0.2989 * r + 0.5870 * g + 0.1140 * b
                    gray.append(gray_scale)
                    
                else:
                    gray.append(j[i])
    #         print(gray)
    #         return j
            return dict(zip(j.keys(),gray))              


In [4]:
# inputarg = preprocess('C:/Users/DELL/Desktop/result_1/results_images/output_final/',3,shuffle =False)
# rescaled_output1 = inputarg.edge_detection()

# # (10,80),(80,80),(80,200),(10,200)
# print(rescaled_output1)


{'1_ir': array([[ 23,  25,  25, ..., 255, 171, 170],
       [ 11,  14,  16, ...,   3, 171, 170],
       [  5,   9,  11, ..., 253, 170, 171],
       ...,
       [354, 352, 351, ..., 239, 153, 153],
       [124, 124, 124, ..., 154, 156, 157],
       [126, 126, 126, ..., 157, 158, 160]]), '2_ir': array([[234, 344, 239, ...,  15, 161, 167],
       [253, 253,  17, ...,   9, 160, 164],
       [ 47,  31,  31, ...,   9, 160, 164],
       ...,
       [233, 238, 348, ..., 225, 182, 181],
       [175, 177, 181, ..., 187, 188, 188],
       [184, 184, 187, ..., 191, 192, 194]]), '3_ir': array([[356, 252, 356, ..., 252,  78,  78],
       [354, 359, 252, ..., 254,  78,  78],
       [176, 322, 322, ...,   9,  78,  78],
       ...,
       [354, 344, 334, ...,  29, 140, 143],
       [160, 160, 156, ..., 130, 136, 136],
       [160, 157, 160, ..., 127, 133, 133]])}
